# SB_CNN_SED Example

In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
import glob
import numpy as np
import argparse

sys.path.append('../')
from dcase_models.utils.files import load_json, mkdir_if_not_exists
from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.datasets import URBAN_SED
from dcase_models.data.features import MelSpectrogram
from dcase_models.model.models import SB_CNN_SED
from dcase_models.data.scaler import Scaler

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define parameters

In [43]:
# features parameters
sequence_time = 1.0
sequence_hop_time = 1.0
audio_hop = 640
audio_win = 1024
n_fft = 1024
sr = 22050
mel_bands = 128

# normalizer
normalizer = 'minmax'

# train parameters
early_stopping = 10
epochs = 5
considered_improvement = 0
learning_rate = 0.001
batch_size = 64
verbose = 1
optimizer = 'Adam'

# dataset parameters
dataset_path = '../../../../data/pzinemanas/URBAN-SED_v2.0.0'

## Init Feature Extractor and Data Generator

In [44]:
# Init Feature Extractor
feature_extractor = MelSpectrogram(sequence_time=sequence_time, 
                                   sequence_hop_time=sequence_hop_time, 
                                   audio_win=audio_win, 
                                   audio_hop=audio_hop, 
                                   n_fft=n_fft, 
                                   sr=sr, mel_bands=mel_bands)

print(feature_extractor.get_features_shape())

# Init Data Generator
dataset = URBAN_SED(dataset_path, sequence_hop_time=sequence_hop_time)

(11, 32, 128)


In [47]:
data_generator = DataGenerator(dataset, feature_extractor)

## Extract Features

In [ ]:
data_generator.extract_features()
print('Done!')

## Load data

In [27]:
print('Loading data... ')
data_generator.load_data()
print('Done!')

Loading data... 
fold: [############################################################] 3/3
Done!


## Fit scaler

In [28]:
fold_test = 'test'

X_train, Y_train, X_val, Y_val = data_generator.get_data_for_training(fold_test, 
                                                                      evaluation_mode='train-validate-test')
scaler = Scaler(normalizer=normalizer)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [29]:
print(X_train.shape, Y_train.shape, X_val[0].shape, Y_val[0].shape)

(66000, 128, 128) (66000, 10) (11, 128, 128) (11, 10)


## Init Model

In [46]:
n_frames_cnn = X_train.shape[1]
n_freq_cnn = X_train.shape[2]
n_classes = Y_train.shape[1]
print(n_frames_cnn, n_freq_cnn, n_classes)
metrics = ['F1', 'ER']

model_container = SB_CNN_SED(model=None, model_path=None, n_classes=n_classes, 
                             n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn,
                             metrics=metrics)

model_container.model.summary()

32 128 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 32, 128)           0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 32, 128, 1)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 124, 64)       1664      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 62, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 14, 62, 64)        256       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 10, 58, 64)        102464    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 29, 64)         0         


## Train model

In [34]:
exp_folder = './'

train_arguments = {'early_stopping': early_stopping,
                  'epochs': epochs,
                  'considered_improvement': considered_improvement,
                  'learning_rate': learning_rate,
                  'batch_size': batch_size,
                  'verbose': verbose,
                  'optimizer': optimizer}

model_container.train(X_train, Y_train, X_val, Y_val, weights_path=exp_folder, **train_arguments)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
66000/66000 [==============================] - 43s 658us/step - loss: 2.9484
F1 = 0.2196, ER = 7.1058 - Best val F1: 0.2196
                  (IMPROVEMENT, saving)

Epoch 2/5
 6720/66000 [==>...........................] - ETA: 35s - loss: 2.9018

KeyboardInterrupt: 

## Evaluate Model

In [23]:
# Load best_weights
model_container.load_model_weights(exp_folder)

# Test model
X_test, Y_test = data_generator.get_data_for_testing(fold_test)
X_test = scaler.transform(X_test)
results = model_container.evaluate(X_test, Y_test)

print(results['accuracy'])

0.75
